In [1]:
# initial setup for any notebook

%load_ext autoreload
%autoreload 2
import sys
sys.path.append("/home/yarcoh/projects/thesis-code4") # go to parent dir

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from src.general_methods import multi_run
from src.optics.channel_simulation import ChannelSimulator
from src.optics.split_step_fourier import SplitStepFourier
from src.general_methods.visualizer import Visualizer

# Deep Learning research

## BER vs normalization factor

In [ ]:
# test the impact of ber vs normalization factor for multiple n_symbols
ssf = SplitStepFourier(
    b2=-20e-27,
    gamma=0.003,
    t0=125e-12,
    z_n=1000e3,
    dt=1,
    h=200
)
cs = ChannelSimulator(m_qam=16,dt=1, verbose=False, ssf=ssf)

num_realisations = 3
# us_vec = multi_run.create_us_vec(n_steps=16, min_u=-3.5, max_u=1)
us_vec = np.linspace(start=0.001,stop=0.1,num=16)
N_symbols_vec = np.array([64, 128, 256, 512])

with np.printoptions(precision=2):
    print(f'normalizing factors are: {us_vec}')

    bers_vec, legends = [], []
    for n_sym in N_symbols_vec:
        cs.num_symbols = n_sym
        bers, errs = multi_run.ber_vs_us(cs, us_vec, n_realisations=num_realisations)
        print(f'{n_sym} symbols: errors found = {errs} / [{cs.length_of_msg * num_realisations} '
              f'= {cs.length_of_msg} bits * {num_realisations} realisations]')
        bers_vec.append(bers)
        legends.append(f'{n_sym} symbols')

In [ ]:
Visualizer.plot_bers(us_vec, bers_vec, legends)